In [2]:
## 分析同源基因对TPM的表达差异
tissues <- c('anther', 'cotyledon','hypocotyl', 'leaf', 'ovule', 'petal', 'radicle', 'root', 'stem', 'stigma', 'fiber_10DPA', 'fiber_20DPA')

for (t in tissues) {
    input_f = paste("G:/Billfish/J668_multip_tissue_3D-genome/new_RNA_seq/homo_gene2/gene_bias/TPM/", t, "/", t, "_pairs_tpm.txt", sep="")
    output_f = paste("G:/Billfish/J668_multip_tissue_3D-genome/new_RNA_seq/homo_gene2/gene_bias/TPM/", t, "/", t, "_pairs_tpm_t-test.txt", sep="")
    df = read.table(input_f, sep="\t", header=T)
    n <- c(1:nrow(df))
    t_test <- rep(NA, nrow(df)) # 建立t检验的p值空间向量
    p.w <- rep(NA, nrow(df)) # 秩和检验的p值空间向量
    tpm_fc <- rep(NA, nrow(df)) # 差异倍数
    for (i in n)
    {
        #print(as.numeric(df[i, 3:5]))
        #print(as.numeric(df[i, 3:4]))
        #print(rowMeans(df[i, 3:4], na.rm=TRUE))

        # tpm_fc[i] <- (rowMeans(df[i, 3:4], na.rm=TRUE)+0.001)/(rowMeans(df[i, 5:6], na.rm=TRUE)+0.001)
        # t_test[i] <- t.test(df[i,3:4], df[i,5:6])$p.value

        tpm_fc[i] <- (rowMeans(df[i, 3:5], na.rm=TRUE)+0.001)/(rowMeans(df[i, 6:8], na.rm=TRUE)+0.001)
        t_test[i] <- t.test(df[i, 3:5], df[i, 6:8])$p.value
        
        #tpm_fc[i] <- 
        #p.w[i] <- wilcox.test(as.numeric(df[i, 3:5]), as.numeric(df[i, 6:8]), exact=FALSE)$p.value   
    }
    fdr <- p.adjust(t_test, method='fdr', length(t_test))
    df_result <- cbind(df, tpm_fc, t_test, fdr)
    write.table(df_result, file=output_f, sep="\t", row.names=F)
}

In [9]:
## 随机选择1000K个基因，并得到其WGCNA 互作强度
import pandas as pd
df = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/Gh_gene_position.bed", sep="\t")
df_WGCNA = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/WGCNA/J668_all_tissue_005.edges.txt", sep="\t")
df_1K = df.sample(100)[['GeneId']].copy()
df_1K.columns = ['fromNode']
df_a = pd.merge(df_WGCNA, df_1K, how='inner', on=['fromNode']).copy()
df_1K.columns = ['toNode']
df_b = pd.merge(df_WGCNA, df_1K, how='inner', on=['toNode']).copy()
df_result = pd.concat([df_a, df_b], axis=0)
df_result.drop_duplicates(inplace=True)
print(df_result)

             fromNode           toNode    weight
0     Ghir_A01G004980  Ghir_A01G005700  0.054021
1     Ghir_A01G004980  Ghir_A01G016550  0.064616
2     Ghir_A01G004980  Ghir_A01G019680  0.074318
3     Ghir_A01G004980  Ghir_A01G019690  0.067586
4     Ghir_A01G004980  Ghir_A02G002460  0.060104
...               ...              ...       ...
9009  Ghir_A10G022380  Ghir_D09G018040  0.098983
9010  Ghir_D03G012060  Ghir_D09G018040  0.056544
9011  Ghir_A11G023430  Ghir_D11G020970  0.095210
9012  Ghir_A11G034790  Ghir_D12G010150  0.053675
9013  Ghir_A12G020940  Ghir_D04G003380  0.051400

[17470 rows x 3 columns]


In [16]:
## 给每个基因打上区间和hic-pro tags标记
import pandas as pd
df_abs = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/Gh_3K_abs.bed", sep="\t", names = ['chrs', 'start', 'end', 'tags'])
df_gene = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/Gh_gene_position.bed", sep="\t")
chrom = ['Ghir_A01', 'Ghir_A02', 'Ghir_A03', 'Ghir_A04', 'Ghir_A05', 'Ghir_A06', 'Ghir_A07', 'Ghir_A08', 'Ghir_A09', 'Ghir_A10', 'Ghir_A11', 'Ghir_A12', 'Ghir_A13', 
         'Ghir_D01', 'Ghir_D02', 'Ghir_D03', 'Ghir_D04', 'Ghir_D05', 'Ghir_D06', 'Ghir_D07', 'Ghir_D08', 'Ghir_D09', 'Ghir_D10', 'Ghir_D11', 'Ghir_D12', 'Ghir_D13']
## 给每个基因打上标签
def gene_tags(df_tags, df_gene):
    df_return = pd.DataFrame()
    for index, row in df_tags.iterrows():
        s, e, t = row['start'], row['end'], row['tags']
        df_gene_tags = df_gene.query("start<=@e & end>=@s").copy()
        df_gene_tags['tags'] = [t]*df_gene_tags.shape[0]
        df_return = pd.concat([df_return, df_gene_tags], axis=0)
        #print(df_return)
    df_return.drop_duplicates(keep='first', subset=['GeneId'], inplace=True)
    return df_return
df_result = pd.DataFrame()
for c in chrom:
    print(c)
    df_gene_chr = df_gene.query("chrs==@c").copy()
    df_abs_chr = df_abs.query("chrs==@c").copy()
    df_tmp = gene_tags(df_abs_chr, df_gene_chr)
    df_result = pd.concat([df_result, df_tmp], axis=0)



Ghir_A01
Ghir_A02
Ghir_A03
Ghir_A04
Ghir_A05
Ghir_A06
Ghir_A07
Ghir_A08
Ghir_A09
Ghir_A10
Ghir_A11
Ghir_A12
Ghir_A13
Ghir_D01
Ghir_D02
Ghir_D03
Ghir_D04
Ghir_D05
Ghir_D06
Ghir_D07
Ghir_D08
Ghir_D09
Ghir_D10
Ghir_D11
Ghir_D12
Ghir_D13


In [20]:
print(df_result.shape[0])
df_result2 = df_result.drop_duplicates(keep='first', subset=['GeneId'])
df_result2.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/RNA_seq/Gh_3K_geneid.bed", sep="\t", index=False)

151518
